# מפגש 5 – מתזמר רב-סוכנים

מדגים צינור פשוט של שני סוכנים (חוקר -> עורך) באמצעות Foundry Local.


### הסבר: התקנת תלותים
מתקין את `foundry-local-sdk` ו-`openai` הנדרשים לגישה למודל מקומי ולהשלמות שיחה. אידמפוטנטי.


# תרחיש
מיישם תבנית אורקסטרטור מינימלית עם שני סוכנים:
- **סוכן חוקר** אוסף נקודות עובדתיות תמציתיות
- **סוכן עורך** משכתב לצורך בהירות מנהלית

מדגים זיכרון משותף לכל סוכן, העברת פלט ביניים באופן רציף ופונקציית צינור פשוטה. ניתן להרחבה לתפקידים נוספים (לדוגמה, מבקר, מאמת) או לענפים מקבילים.

**משתני סביבה:**
- `FOUNDRY_LOCAL_ALIAS` - מודל ברירת המחדל לשימוש (ברירת מחדל: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - מודל הסוכן הראשי (מחליף את ALIAS)
- `AGENT_MODEL_EDITOR` - מודל סוכן העורך (ברירת מחדל: ראשי)

**הפניה ל-SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**איך זה עובד:**
1. **FoundryLocalManager** מפעיל באופן אוטומטי את שירות Foundry Local
2. מוריד ומטעין את המודל שצוין (או משתמש בגרסה שמורה במטמון)
3. מספק נקודת קצה תואמת OpenAI לצורך אינטראקציה
4. כל סוכן יכול להשתמש במודל שונה למשימות מיוחדות
5. לוגיקת ניסיון חוזר מובנית מטפלת בכשלים זמניים בצורה חלקה

**תכונות עיקריות:**
- ✅ גילוי שירות ואתחול אוטומטי
- ✅ ניהול מחזור חיים של מודלים (הורדה, שמירה במטמון, טעינה)
- ✅ תאימות ל-SDK של OpenAI עבור API מוכר
- ✅ תמיכה בריבוי מודלים להתמחות סוכנים
- ✅ טיפול שגיאות חזק עם לוגיקת ניסיון חוזר
- ✅ הסקה מקומית (ללא צורך ב-API בענן)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### הסבר: ייבוא מרכזי והקלדת סוגים  
מוצגים dataclasses לאחסון הודעות סוכן ורמזי הקלדה לשם בהירות. ייבוא מנהל Foundry Local + לקוח OpenAI לפעולות סוכן עתידיות.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### הסבר: אתחול מודל (תבנית SDK)
משתמש ב-SDK המקומי של Foundry Python לניהול מודלים חזק:
- **FoundryLocalManager(alias)** - מפעיל אוטומטית את השירות וטוען מודל לפי כינוי
- **get_model_info(alias)** - ממפה כינוי למזהה מודל ספציפי
- **manager.endpoint** - מספק נקודת קצה לשירות עבור לקוח OpenAI
- **manager.api_key** - מספק מפתח API (אופציונלי לשימוש מקומי)
- תומך במודלים נפרדים עבור סוכנים שונים (ראשי מול עורך)
- לוגיקת ניסיון מובנית עם גיבוי אקספוננציאלי לשיפור עמידות
- אימות חיבור כדי להבטיח שהשירות מוכן

**תבנית SDK מרכזית:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**ניהול מחזור חיים:**
- מנהלים נשמרים גלובלית לניקוי נכון
- כל סוכן יכול להשתמש במודל שונה להתמחות
- גילוי שירות אוטומטי וטיפול בחיבורים
- ניסיון מחדש בצורה חלקה עם גיבוי אקספוננציאלי במקרה של כשל

זה מבטיח אתחול נכון לפני תחילת תזמור הסוכנים.

**עיון:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### הסבר: מחלקות Agent ו-Memory
מגדיר את `AgentMsg` הקליל עבור רשומות זיכרון ואת `Agent` שמקיף:
- **תפקיד מערכת** - האישיות וההוראות של הסוכן
- **היסטוריית הודעות** - שומרת על הקשר השיחה
- **שיטת act()** - מבצעת פעולות עם טיפול נכון בשגיאות

הסוכן יכול להשתמש במודלים שונים (ראשי מול עורך) ושומר הקשר מבודד לכל סוכן. תבנית זו מאפשרת:
- שמירת זיכרון בין פעולות
- הקצאת מודל גמישה לכל סוכן
- בידוד שגיאות והתאוששות
- שרשור ותזמור פשוטים


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### הסבר: צינור מתוזמר
יוצר שני סוכנים מתמחים:
- **חוקר**: משתמש במודל הראשי, אוסף מידע עובדתי
- **עורך**: יכול להשתמש במודל נפרד (אם מוגדר), משפר ומשכתב

הפונקציה `pipeline`:
1. החוקר אוסף מידע גולמי
2. העורך משפר ומעצב את המידע לתוצר מוכן לביצוע
3. מחזירה גם את התוצאות הביניים וגם את התוצאה הסופית

תבנית זו מאפשרת:
- התמחות מודלים (מודלים שונים לתפקידים שונים)
- שיפור איכות באמצעות עיבוד רב-שלבי
- מעקב אחר תהליך שינוי המידע
- הרחבה קלה לסוכנים נוספים או עיבוד מקבילי


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### הסבר: ביצוע צינור ותוצאות
מבצע את צינור הסוכנים המרובים על שאלה בנושא תאימות + זמן תגובה כדי להדגים:
- שינוי מידע רב-שלבי
- התמחות ושיתוף פעולה בין סוכנים
- שיפור איכות הפלט באמצעות עידון
- עקיבות (שימור פלטים ביניים וסופיים)

**מבנה התוצאה:**
- `question` - השאלה המקורית של המשתמש
- `research` - פלט מחקר גולמי (נקודות עובדתיות)
- `final` - סיכום מנהלים מעודן
- `models` - אילו מודלים שימשו בכל שלב

**רעיונות להרחבה:**
1. הוספת סוכן מבקר לביקורת איכות
2. יישום סוכני מחקר מקבילים להיבטים שונים
3. הוספת סוכן מאמת לבדיקה עובדתית
4. שימוש במודלים שונים לרמות מורכבות שונות
5. יישום לולאות משוב לשיפור איטרטיבי


### מתקדם: תצורת סוכן מותאמת אישית

נסו להתאים את התנהגות הסוכן על ידי שינוי משתני סביבה לפני הפעלת תא האתחול:

**מודלים זמינים:**
- השתמשו ב-`foundry model ls` במסוף כדי לראות את כל המודלים הזמינים
- דוגמאות: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, וכו'.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
